In [10]:
import pandas as pd
import owlready2 as owlr 
#https://pythonhosted.org/Owlready/
import rdflib
from collections import Counter

In [2]:
onto = owlr.get_ontology("datamodels.owl")
onto.load()

get_ontology("http://www.semanticweb.org/tomk/ontologies/2022/9/datamodel#")

In [3]:
dir(onto)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abbreviate',
 '_add_annotation_axiom',
 '_add_data_triple_raw_spod',
 '_add_data_triple_raw_spodsd',
 '_add_data_triple_spod',
 '_add_obj_triple_raw_spo',
 '_add_obj_triple_spo',
 '_base_iri',
 '_bnodes',
 '_del_annotation_axiom',
 '_del_data_triple_raw_spod',
 '_del_data_triple_raw_spodsd',
 '_del_data_triple_spod',
 '_del_list',
 '_del_obj_triple_raw_spo',
 '_del_obj_triple_spo',
 '_destroy_cached_entities',
 '_entity_destroyed',
 '_get_annotation_axioms',
 '_get_data_triple_sp_od',
 '_get_data_triples_s_pod',
 '_get_data_triples_sp_od',
 '_get_data_triples_sp

In [6]:
individuals = [(i, i.is_a.first()) for i in onto.individuals()]
serializations = [i[0] for i in individuals if i[1] == onto.Serialization]
for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping]
    for m in mappings:
        print ("\t", m, m.is_a)

datamodels.DomainModelClassEntityRecord
	 datamodels.AttributeMapping [datamodels.Mapping]
	 datamodels.ClassMapping [datamodels.Mapping]
	 datamodels.DomainMapping [datamodels.Mapping]
	 datamodels.ModelMapping [datamodels.Mapping]
	 datamodels.hasAttributeMapping [datamodels.Mapping]
	 datamodels.hasClassMapping [datamodels.Mapping]
	 datamodels.hasDataType [datamodels.Mapping]
	 datamodels.hasModelMapping [datamodels.Mapping]
	 datamodels.hasModelTypeMapping [datamodels.Mapping]


In [7]:
with onto:
    owlr.sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmpmdu4vifc
* Owlready2 * HermiT took 1.145570993423462 seconds
* Owlready * Equivalenting: datamodels.Identifier datamodels.r_Identifier
* Owlready * Equivalenting: datamodels.r_Identifier datamodels.Identifier
* Owlready * Equivalenting: datamodels.PartialIdentifier datamodels.r_PartialIdentifier
* Owlready * Equivalenting: datamodels.r_PartialIdentifier datamodels.PartialIdentifier
* Owlready * Reparenting datamodels.r_Identifier: {owl.ObjectProperty} => {datamodels.Attribute}
* Owlready * Reparenting datamodels.hasModelTypeMapping: {datamodels.Mapping} => {datamodels.PropertyMapping}
* Owlready * Reparenting datamodels.hasDataType: {datamodels.Mapping} => {datamodels.PropertyMa

In [11]:
mapped_things = [(c,c.MappingKey) for c in onto.classes() if len(c.MappingKey) > 0] + \
                [(p, p.MappingKey) for p in onto.properties() if len(p.MappingKey) > 0]
mappable_things = {v[0]:k for k,v in mapped_things}
assert all([v==1 for v in Counter([b for c in mapped_things for b in c[1]]).values()])
del mapped_things
mappable_things

{'Attribute': datamodels.Attribute,
 'Class': datamodels.Class,
 'Context': datamodels.Context,
 'DataModel': datamodels.DataModel,
 'DataType': datamodels.DataType,
 'DataTypeSchema': datamodels.DataTypeSchema,
 'Mapping': datamodels.Mapping,
 'Domain': datamodels.ModelDomain,
 'Relationship': datamodels.Relationship,
 'Serialization': datamodels.Serialization,
 'ModelTypeCode': datamodels.ModelTypeCode,
 'hasDataType': datamodels.AttributeHasDataType,
 'hasAttribute': datamodels.ClassHasAttribute,
 'hasClass': datamodels.ModelContainsClass,
 'hasModel': datamodels.ModelDomainContainsModel,
 'hasModelType': datamodels.ModelHasModelTypeCode}

In [13]:
data_row = { "Business Domain" : "Core", 
             "Model" : "Principal Interactions", 
           "ModelType" : "Conceptual", 
           "Entity" : "Person", 
           "Attribute" : "PersonID", 
           "DataType" : "xsd:string"}

In [26]:
for s in serializations:
    print(s)
    mappings = [m for m in s.SerializationContainsMapping]
    serials = {}
    for m in mappings:
        print (m)
        s_key = m.SerializationLabel.first()
        i_key = m.MappingLabel.first()
        if s_key not in serials.keys():
            serials[s_key]=[]
        if m.is_a.first() == onto.EntityMapping:
            o_range = m.MappingRange.first()
            serials[s_key].append((data_row[s_key], "rdf:type", mappable_things[o_range]))
    
        elif m.is_a.first() == onto.PropertyMapping:
            s_domain = m.MappingDomain.first()
            o_range = m.MappingRange.first()
            serials[s_key].append((i_key,  mappable_things[s_domain], mappable_things[i_key], mappable_things[o_range], data_row[s_key] ))
        else:
            print(m.is_a)
            print(s_key)
            assert False

for v in serials.values():
    for u in v:
        if u[1]=="rdf:type":
            print(u)

print()

for v in serials.values():
    for u in v:
        if u[1]!="rdf:type":
            print(u)



datamodels.DomainModelClassEntityRecord
datamodels.AttributeMapping
datamodels.ClassMapping
datamodels.DomainMapping
datamodels.ModelMapping
datamodels.hasAttributeMapping
datamodels.hasClassMapping
datamodels.hasDataType
datamodels.hasModelMapping
datamodels.hasModelTypeMapping
('PersonID', 'rdf:type', datamodels.Attribute)
('Person', 'rdf:type', datamodels.Class)
('Core', 'rdf:type', datamodels.ModelDomain)
('Principal Interactions', 'rdf:type', datamodels.DataModel)

('hasAttribute', datamodels.Class, datamodels.ClassHasAttribute, datamodels.Attribute, 'PersonID')
('hasClass', datamodels.DataModel, datamodels.ModelContainsClass, datamodels.Class, 'Person')
('hasModel', datamodels.ModelDomain, datamodels.ModelDomainContainsModel, datamodels.DataModel, 'Principal Interactions')
('hasDataType', datamodels.Attribute, datamodels.AttributeHasDataType, datamodels.DataType, 'xsd:string')
('hasModelType', datamodels.DataModel, datamodels.ModelHasModelTypeCode, datamodels.ModelTypeCode, 'Conc